# Practical example. Audiobooks

## Problem

You are given data from an Audiobook app. Logically, it relates only to the audio versions of books. Each customer in the database has made a purchase at least once, that's why he/she is in the database. We want to create a machine learning algorithm based on our available data that can predict if a customer will buy again from the Audiobook company.

The main idea is that if a customer has a low probability of coming back, there is no reason to spend any money on advertizing to him/her. If we can focus our efforts ONLY on customers that are likely to convert again, we can make great savings. Moreover, this model can identify the most important metrics for a customer to come back again. Identifying new customers creates value and growth opportunities.

You have a .csv summarizing the data. There are several variables: Customer ID, Book length in mins_avg (average of all purchases), Book length in minutes_sum (sum of all purchases), Price Paid_avg (average of all purchases), Price paid_sum (sum of all purchases), Review (a Boolean variable), Review (out of 10), Total minutes listened, Completion (from 0 to 1), Support requests (number), and Last visited minus purchase date (in days).

So these are the inputs (excluding customer ID, as it is completely arbitrary. It's more like a name, than a number).

The targets are a Boolean variable (so 0, or 1). We are taking a period of 2 years in our inputs, and the next 6 months as targets. So, in fact, we are predicting if: based on the last 2 years of activity and engagement, a customer will convert in the next 6 months. 6 months sounds like a reasonable time. If they don't convert after 6 months, chances are they've gone to a competitor or didn't like the Audiobook way of digesting information. 

The task is simple: create a machine learning algorithm, which is able to predict if a customer will buy again. 

This is a classification problem with two classes: won't buy and will buy, represented by 0s and 1s. 

## Create the machine learning algorithm



### Import the relevant libraries

In [1]:
import numpy as np
import tensorflow as tf

### Data

In [2]:
# let's create a temporary variable npz, where we will store each of the three Audiobooks datasets
npz = np.load('Audiobooks_data_train.npz')

# we extract the inputs using the keyword under which we saved them
# to ensure that they are all floats, let's also take care of that
train_inputs = npz['inputs'].astype(float)
# targets must be int because of sparse_categorical_crossentropy (we want to be able to smoothly one-hot encode them)
train_targets = npz['targets'].astype(int)

# we load the validation data in the temporary variable
npz = np.load('Audiobooks_data_validation.npz')
# we can load the inputs and the targets in the same line
validation_inputs, validation_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

# we load the test data in the temporary variable
npz = np.load('Audiobooks_data_test.npz')
# we create 2 variables that will contain the test inputs and the test targets
test_inputs, test_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

### Model 1
Outline, optimizers, loss, early stopping and training

In [3]:
# Set the input and output sizes
input_size = 10
output_size = 2
hidden_layer_size = 15
    
# define how the model will look like
model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(50, activation='relu'), # 2nd hidden layer
    #tf.keras.layers.Dense(hidden_layer_size, activation='relu',kernel_initializer='he_uniform'),
    
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

#Choose the optimizer and the loss function
custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=custom_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Training
# That's where we train the model we have built.

batch_size = 100

max_epochs = 100

# set an early stopping mechanism
# let's set patience=2, to be a bit tolerant against random validation loss increases
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,
          train_targets, 
          batch_size=batch_size, 
          epochs=max_epochs, 
          callbacks=[early_stopping], 
          validation_data=(validation_inputs, validation_targets), 
          verbose = 2 # making sure we get enough information about the training process
          )  

print('\n\n')
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Metal device set to: Apple M1
Epoch 1/100


2022-09-27 11:58:41.925773: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-27 11:58:41.926874: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-09-27 11:58:42.034956: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-09-27 11:58:42.283829: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


36/36 - 1s - loss: 0.6380 - accuracy: 0.6049 - val_loss: 0.5796 - val_accuracy: 0.6890 - 945ms/epoch - 26ms/step
Epoch 2/100


2022-09-27 11:58:42.944212: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


36/36 - 0s - loss: 0.5272 - accuracy: 0.7139 - val_loss: 0.5165 - val_accuracy: 0.7293 - 192ms/epoch - 5ms/step
Epoch 3/100
36/36 - 0s - loss: 0.4627 - accuracy: 0.7611 - val_loss: 0.4753 - val_accuracy: 0.7360 - 178ms/epoch - 5ms/step
Epoch 4/100
36/36 - 0s - loss: 0.4224 - accuracy: 0.7837 - val_loss: 0.4562 - val_accuracy: 0.7494 - 183ms/epoch - 5ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3985 - accuracy: 0.7896 - val_loss: 0.4367 - val_accuracy: 0.7472 - 175ms/epoch - 5ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3838 - accuracy: 0.7980 - val_loss: 0.4274 - val_accuracy: 0.7494 - 180ms/epoch - 5ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3729 - accuracy: 0.7980 - val_loss: 0.4178 - val_accuracy: 0.7651 - 173ms/epoch - 5ms/step
Epoch 8/100
36/36 - 0s - loss: 0.3656 - accuracy: 0.8022 - val_loss: 0.4125 - val_accuracy: 0.7718 - 177ms/epoch - 5ms/step
Epoch 9/100
36/36 - 0s - loss: 0.3594 - accuracy: 0.8027 - val_loss: 0.4079 - val_accuracy: 0.7718 - 206ms/epoch - 6ms/step
Epoch 10/100
36/36 -

## Model 2

In [4]:
npz = np.load('Audiobooks_data_train.npz')

train_inputs = npz['inputs'].astype(float)
# targets must be int because of sparse_categorical_crossentropy (we want to be able to smoothly one-hot encode them)
train_targets = npz['targets'].astype(int)

# we load the validation data in the temporary variable
npz = np.load('Audiobooks_data_validation.npz')
# we can load the inputs and the targets in the same line
validation_inputs, validation_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

# we load the test data in the temporary variable
npz = np.load('Audiobooks_data_test.npz')
# we create 2 variables that will contain the test inputs and the test targets
test_inputs, test_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

In [5]:
# Set the input and output sizes
input_size = 10
output_size = 2

hidden_layer_size = 60
    

model2 = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
 
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.002)
model2.compile(optimizer=custom_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 150

max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model2.fit(train_inputs,
          train_targets, 
          batch_size=batch_size, 
          epochs=max_epochs, 
          callbacks=[early_stopping], 
          validation_data=(validation_inputs, validation_targets), 
          verbose = 2 # making sure we get enough information about the training process
          )  

print('\n\n')
test_loss, test_accuracy = model2.evaluate(test_inputs, test_targets)
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Epoch 1/100


2022-09-27 12:00:20.193243: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


24/24 - 1s - loss: 0.5250 - accuracy: 0.7220 - val_loss: 0.4725 - val_accuracy: 0.7315 - 558ms/epoch - 23ms/step
Epoch 2/100
24/24 - 0s - loss: 0.4079 - accuracy: 0.7840 - val_loss: 0.4226 - val_accuracy: 0.7696 - 133ms/epoch - 6ms/step
Epoch 3/100


2022-09-27 12:00:20.462037: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


24/24 - 0s - loss: 0.3753 - accuracy: 0.7879 - val_loss: 0.3994 - val_accuracy: 0.7875 - 138ms/epoch - 6ms/step
Epoch 4/100
24/24 - 0s - loss: 0.3558 - accuracy: 0.8097 - val_loss: 0.4025 - val_accuracy: 0.7875 - 128ms/epoch - 5ms/step
Epoch 5/100
24/24 - 0s - loss: 0.3487 - accuracy: 0.8128 - val_loss: 0.3795 - val_accuracy: 0.7808 - 128ms/epoch - 5ms/step
Epoch 6/100
24/24 - 0s - loss: 0.3435 - accuracy: 0.8058 - val_loss: 0.4147 - val_accuracy: 0.7897 - 128ms/epoch - 5ms/step
Epoch 7/100
24/24 - 0s - loss: 0.3416 - accuracy: 0.8178 - val_loss: 0.3762 - val_accuracy: 0.7740 - 135ms/epoch - 6ms/step
Epoch 8/100
24/24 - 0s - loss: 0.3391 - accuracy: 0.8136 - val_loss: 0.3814 - val_accuracy: 0.7785 - 127ms/epoch - 5ms/step
Epoch 9/100
24/24 - 0s - loss: 0.3298 - accuracy: 0.8215 - val_loss: 0.3683 - val_accuracy: 0.7964 - 126ms/epoch - 5ms/step
Epoch 10/100
24/24 - 0s - loss: 0.3294 - accuracy: 0.8203 - val_loss: 0.3666 - val_accuracy: 0.7987 - 126ms/epoch - 5ms/step
Epoch 11/100
24/24 

## Model 3

In [8]:
npz = np.load('Audiobooks_data_train.npz')

train_inputs = npz['inputs'].astype(float)
# targets must be int because of sparse_categorical_crossentropy (we want to be able to smoothly one-hot encode them)
train_targets = npz['targets'].astype(int)

# we load the validation data in the temporary variable
npz = np.load('Audiobooks_data_validation.npz')
# we can load the inputs and the targets in the same line
validation_inputs, validation_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

# we load the test data in the temporary variable
npz = np.load('Audiobooks_data_test.npz')
# we create 2 variables that will contain the test inputs and the test targets
test_inputs, test_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

In [9]:
input_size = 10
output_size = 2

hidden_layer_size = 50

model3 = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
    
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
    
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
    tf.keras.layers.Dense(hidden_layer_size, activation='relu',kernel_initializer='he_uniform'),
 
    tf.keras.layers.Dense(output_size, activation='softmax') 
])


custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model3.compile(optimizer=custom_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 50

max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=3)

model3.fit(train_inputs, 
          train_targets, 
          batch_size=batch_size, 
          epochs=max_epochs, 
          callbacks=[early_stopping],
          validation_data=(validation_inputs, validation_targets),
          verbose = 2 # making sure we get enough information about the training process
          )  

print('\n\n')
test_loss, test_accuracy = model3.evaluate(test_inputs, test_targets)
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Epoch 1/100


2022-09-27 12:02:45.260434: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


72/72 - 1s - loss: 0.5120 - accuracy: 0.7153 - val_loss: 0.4160 - val_accuracy: 0.7673 - 988ms/epoch - 14ms/step
Epoch 2/100


2022-09-27 12:02:45.933486: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


72/72 - 1s - loss: 0.3752 - accuracy: 0.7907 - val_loss: 0.4073 - val_accuracy: 0.7450 - 555ms/epoch - 8ms/step
Epoch 3/100
72/72 - 1s - loss: 0.3520 - accuracy: 0.8069 - val_loss: 0.4022 - val_accuracy: 0.7673 - 530ms/epoch - 7ms/step
Epoch 4/100
72/72 - 0s - loss: 0.3413 - accuracy: 0.8069 - val_loss: 0.4012 - val_accuracy: 0.7852 - 492ms/epoch - 7ms/step
Epoch 5/100
72/72 - 0s - loss: 0.3363 - accuracy: 0.8136 - val_loss: 0.3858 - val_accuracy: 0.7785 - 499ms/epoch - 7ms/step
Epoch 6/100
72/72 - 0s - loss: 0.3421 - accuracy: 0.8086 - val_loss: 0.3967 - val_accuracy: 0.7785 - 482ms/epoch - 7ms/step
Epoch 7/100
72/72 - 0s - loss: 0.3414 - accuracy: 0.8047 - val_loss: 0.3843 - val_accuracy: 0.7785 - 495ms/epoch - 7ms/step
Epoch 8/100
72/72 - 0s - loss: 0.3298 - accuracy: 0.8159 - val_loss: 0.4045 - val_accuracy: 0.7696 - 497ms/epoch - 7ms/step
Epoch 9/100
72/72 - 1s - loss: 0.3271 - accuracy: 0.8198 - val_loss: 0.3772 - val_accuracy: 0.7740 - 536ms/epoch - 7ms/step
Epoch 10/100
72/72 -

In [30]:
import tensorflow as tf
print(tf.__version__)

2.9.1


Thanks to 365 Data Science, thats where I learned most of the content I used to create these models.